<a href="https://colab.research.google.com/github/darkchoco171/jeys/blob/main/Klasifikasi_Sleep_Disorder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Library**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# **Memuat** **dataset**

In [19]:
from google.colab import drive
drive.mount = ('/content/Sleep_health_and_lifestyle_dataset.csv')

In [30]:
df=pd.DataFrame('/content/Sleep_health_and_lifestyle_dataset.csv');

Dataset berhasil dimuat menggunakan pd.read_csv().


In [31]:
print("Info DataFrame:")
df.info()

print("\n5 Baris Pertama DataFrame:")
df.head()

Info DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Gender                  374 non-null    object
 1   Age                     374 non-null    int64 
 2   Occupation              374 non-null    object
 3   Sleep Duration          374 non-null    int64 
 4   Quality of Sleep        374 non-null    int64 
 5   Physical ActivityLevel  374 non-null    int64 
 6   Stress Level            374 non-null    int64 
 7   BMI Category            374 non-null    object
 8   Systolic BP             374 non-null    int64 
 9   Diastolic BP            374 non-null    int64 
 10  Heart Rate              374 non-null    int64 
 11  Daily Steps             374 non-null    int64 
 12  Sleep Disorder          155 non-null    object
 13  Has Disorder            374 non-null    object
dtypes: int64(9), object(5)
memory usage: 41.0+

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical ActivityLevel,Stress Level,BMI Category,Systolic BP,Diastolic BP,Heart Rate,Daily Steps,Sleep Disorder,Has Disorder
0,Male,27,Software Engineer,6,6,42,6,Overweight,126,83,77,4200,NaN,No
1,Male,28,Doctor,6,6,60,8,Normal,125,80,75,10000,NaN,No
2,Male,28,Doctor,6,6,60,8,Normal,125,80,75,10000,NaN,No
3,Male,28,Sales Representative,6,4,30,8,Obese,140,90,85,3000,Sleep Apnea,Yes
4,Male,28,Sales Representative,6,4,30,8,Obese,140,90,85,3000,Sleep Apnea,Yes


In [65]:
# Membuat target 'Sleep Disorder' berdasarkan kondisi logis sederhana
conditions = [
(df['Stress Level'] > 6) & (df['Quality of Sleep'] < 5),
(df['BMI Category'] == 'Obese') & (df['Systolic BP'] > 135),
(df['Sleep Duration'] < 6.5) & (df['Age'] > 40)]

# Pre-Processing Lanjutan

In [99]:
print("\nMemulai pra-pemrosesan data...")
# Memisahkan fitur (X) dan target (y)
# Memastikan df masih memiliki kolom kategorikal asli sebelum membuat X
X = df.drop('Sleep Disorder', axis=1)
y = df['Sleep Disorder']

print("Kolom di X sebelum One-Hot Encoding:", X.columns.tolist())


Memulai pra-pemrosesan data...
Kolom di X sebelum One-Hot Encoding: ['Gender', 'Age', 'Occupation', 'Sleep Duration', 'Quality of Sleep', 'Physical ActivityLevel', 'Stress Level', 'BMI Category', 'Systolic BP', 'Diastolic BP', 'Heart Rate', 'Daily Steps', 'Has Disorder']


In [100]:
# Mengubah data kategorikal menjadi numerik menggunakan One-Hot Encoding
X = pd.get_dummies(X, columns=['Gender', 'Occupation', 'BMI Category','Has Disorder'], drop_first=True)

In [101]:
# Mengubah label target menjadi numerik
print("\nMelakukan Label Encoding pada target 'Sleep Disorder'...")
label_encoder = LabelEncoder()
# Memastikan LabelEncoder di-fit pada kolom setelah penanganan nilai hilang
y_encoded = label_encoder.fit_transform(df['Sleep Disorder'])
print("Label Encoding target selesai.")


Melakukan Label Encoding pada target 'Sleep Disorder'...
Label Encoding target selesai.


In [102]:
# Menyimpan mapping dari kelas ke angka untuk referensi
class_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}
print("Mapping kelas target: " + str(class_mapping))
print("Pra-pemrosesan selesai.\n")
print("-" * 30)

Mapping kelas target: {0: 'Insomnia', 1: 'Sleep Apnea', 2: nan}
Pra-pemrosesan selesai.

------------------------------


# **Pembagian Data**

In [103]:
print("\nMembagi data menjadi data latih (80%) dan data uji (20%)...")
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)
print(f"Jumlah data latih: {X_train.shape[0]}")
print(f"Jumlah data uji: {X_test.shape[0]}\n")
print("-" * 30)


Membagi data menjadi data latih (80%) dan data uji (20%)...
Jumlah data latih: 299
Jumlah data uji: 75

------------------------------


# **Pemodelan**

In [104]:
print("\nMelatih model Klasifikasi Random Forest...")
# Inisialisasi model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Melatih model menggunakan data latih
model.fit(X_train, y_train)
print("Pelatihan model selesai.\n")
print("-" * 30)


Melatih model Klasifikasi Random Forest...
Pelatihan model selesai.

------------------------------


# **Evaluasi Model**

In [105]:
print("\nMelakukan evaluasi model pada data uji...")
# Membuat prediksi pada data uji
y_pred = model.predict(X_test)


Melakukan evaluasi model pada data uji...


In [106]:
# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)

In [107]:
print(f"\n================ HASIL EVALUASI ================")
print(f"Akurasi Model: {accuracy * 100:.2f}%")
print(f"================================================")


================ HASIL EVALUASI ================
Akurasi Model: 94.67%


In [109]:
# Menampilkan laporan klasifikasi yang lebih detail
print("\nLaporan Klasifikasi (Classification Report):")
# Pastikan target_names adalah list of strings
target_names_list = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_test, y_pred, target_names=target_names_list, zero_division=0))


Laporan Klasifikasi (Classification Report):
              precision    recall  f1-score   support

    Insomnia       0.87      0.87      0.87        15
 Sleep Apnea       0.88      0.88      0.88        16
         nan       1.00      1.00      1.00        44

    accuracy                           0.95        75
   macro avg       0.91      0.91      0.91        75
weighted avg       0.95      0.95      0.95        75



In [110]:
# Menampilkan confusion matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_df = pd.DataFrame(conf_matrix, index=label_encoder.classes_, columns=label_encoder.classes_)
print(conf_matrix_df)


Confusion Matrix:
             Insomnia  Sleep Apnea  NaN
Insomnia           13            2    0
Sleep Apnea         2           14    0
NaN                 0            0   44
